In [1]:
import ast
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyarrow as pa
import pyarrow.parquet as pq

In [25]:
filas = []

with open("output_steam_games.json", "r", encoding="Latin-1") as archivo:
    for linea in archivo:
        try:
            object_json = json.loads(linea)
            filas.append(object_json)
        except json.JSONDecodeError:
            print(f"Error de Fromato Json en la linea {linea}")


df_games = pd.DataFrame(filas)

In [26]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


In [27]:
nulos1 = df_games.isna().sum()
nulos1

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

In [28]:
df_games.drop(columns=["publisher", "title", "url", "early_access", "reviews_url", "specs"], inplace=True)

In [29]:
df_games = df_games.dropna(subset=["id"])

In [30]:
df_games.reset_index(drop=True, inplace=True)

In [15]:
df_games.head(10)

,genres,app_name,release_date,tags,price,id,developer
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.99,761140,Kotoshiro
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",Free To Play,643980,Secret Level SRL
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",Free to Play,670290,Poolians.com
3,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",0.99,767400,彼岸领域
4,NaN,Log Challenge,NaN,"[Action, Indie, Casual, Sports]",2.99,773570,NaN
5,"[Action, Adventure, Simulation]",Battle Royale Trainer,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...",3.99,772540,Trickjump Games Ltd
6,"[Free to Play, Indie, Simulation, Sports]",SNOW - All Access Basic Pass,2018-01-04,"[Free to Play, Indie, Simulation, Sports]",9.99,774276,Poppermost Productions
7,"[Free to Play, Indie, Simulation, Sports]",SNOW - All Access Pro Pass,2018-01-04,"[Free to Play, Indie, Simulation, Sports]",18.99,774277,Poppermost Productions
8,"[Free to Play, Indie, Simulation, Sports]",SNOW - All Access Legend Pass,2018-01-04,"[Free to Play, Indie, Simulation, Sports]",29.99,774278,Poppermost Productions
9,"[Casual, Indie, Racing, Simulation]",Race,2018-01-04,"[Indie, Casual, Simulation, Racing]",NaN,768800,RewindApp


In [31]:
df_games.info()
df_games.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32133 entries, 0 to 32132
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   genres        28851 non-null  object
 1   app_name      32132 non-null  object
 2   release_date  30067 non-null  object
 3   tags          31971 non-null  object
 4   price         30756 non-null  object
 5   id            32133 non-null  object
 6   developer     28835 non-null  object
dtypes: object(7)
memory usage: 1.7+ MB


genres          3282
app_name           1
release_date    2066
tags             162
price           1377
id                 0
developer       3298
dtype: int64

In [32]:
df_games["release_year"] = df_games["release_date"].str.extract(r"(\d{4})")
df_games.drop( columns = "release_date", inplace = True)
df_games.head(5)

,genres,app_name,tags,price,id,developer,release_year
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,"[Strategy, Action, Indie, Casual, Simulation]",4.99,761140,Kotoshiro,2018
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,"[Free to Play, Strategy, Indie, RPG, Card Game...",Free To Play,643980,Secret Level SRL,2018
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,"[Free to Play, Simulation, Sports, Casual, Ind...",Free to Play,670290,Poolians.com,2017
3,"[Action, Adventure, Casual]",弹炸人2222,"[Action, Adventure, Casual]",0.99,767400,彼岸领域,2017
4,NaN,Log Challenge,"[Action, Indie, Casual, Sports]",2.99,773570,NaN,NaN


In [33]:
df_games['genres'] = df_games['genres'].fillna('[]')  # Rellenar los valores faltantes con una lista vacía
df_games['genres'] = df_games['genres'].apply(lambda x: ', '.join(x))  # Convertir la lista de géneros a una cadena separada por comas

#Crear variables ficticias para los géneros
dummy_genres = df_games['genres'].str.get_dummies(', ')

#Concatenar las variables ficticias con el DataFrame original
df_games = pd.concat([df_games, dummy_genres], axis=1)

In [34]:
df_games.head(5)

,genres,app_name,tags,price,id,developer,release_year,Accounting,Action,Adventure,...,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing,[,]
0,"Action, Casual, Indie, Simulation, Strategy",Lost Summoner Kitty,"[Strategy, Action, Indie, Casual, Simulation]",4.99,761140,Kotoshiro,2018,0,1,0,...,0,1,0,0,1,0,0,0,0,0
1,"Free to Play, Indie, RPG, Strategy",Ironbound,"[Free to Play, Strategy, Indie, RPG, Card Game...",Free To Play,643980,Secret Level SRL,2018,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,"Casual, Free to Play, Indie, Simulation, Sports",Real Pool 3D - Poolians,"[Free to Play, Simulation, Sports, Casual, Ind...",Free to Play,670290,Poolians.com,2017,0,0,0,...,0,1,0,1,0,0,0,0,0,0
3,"Action, Adventure, Casual",弹炸人2222,"[Action, Adventure, Casual]",0.99,767400,彼岸领域,2017,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,"[, ]",Log Challenge,"[Action, Indie, Casual, Sports]",2.99,773570,NaN,NaN,0,0,0,...,0,0,0,0,0,0,0,0,1,1


In [35]:
df_games.drop(columns=["[","]","genres"],inplace=True)

In [37]:
df_games.shape

(32133, 28)

In [38]:
df_games.isnull().sum()

app_name                        1
tags                          162
price                        1377
id                              0
developer                    3298
release_year                 2167
Accounting                      0
Action                          0
Adventure                       0
Animation &amp; Modeling        0
Audio Production                0
Casual                          0
Design &amp; Illustration       0
Early Access                    0
Education                       0
Free to Play                    0
Indie                           0
Massively Multiplayer           0
Photo Editing                   0
RPG                             0
Racing                          0
Simulation                      0
Software Training               0
Sports                          0
Strategy                        0
Utilities                       0
Video Production                0
Web Publishing                  0
dtype: int64

In [40]:
df_games.fillna('0', inplace= True)
df_games.isna().sum()

app_name                     0
tags                         0
price                        0
id                           0
developer                    0
release_year                 0
Accounting                   0
Action                       0
Adventure                    0
Animation &amp; Modeling     0
Audio Production             0
Casual                       0
Design &amp; Illustration    0
Early Access                 0
Education                    0
Free to Play                 0
Indie                        0
Massively Multiplayer        0
Photo Editing                0
RPG                          0
Racing                       0
Simulation                   0
Software Training            0
Sports                       0
Strategy                     0
Utilities                    0
Video Production             0
Web Publishing               0
dtype: int64

In [ ]:
df_games['release_year'] = df_games['release_year'].astype(int)

In [ ]:
df_games = df_games.rename(columns= {"id" : "item_id"})

In [44]:
string_price = df_games[df_games["price"].apply(lambda x: isinstance(x, str))]
print (string_price["price"].value_counts())

price
0                                1377
Free                              905
Free to Play                      520
Free To Play                      462
Free Mod                            4
Free Demo                           3
Play for Free!                      2
Third-party                         2
Play Now                            2
Starting at $499.00                 1
Free Movie                          1
Free to Try                         1
Starting at $449.00                 1
Install Theme                       1
Play the Demo                       1
Free HITMAN™ Holiday Pack           1
Play WARMACHINE: Tactics Demo       1
Install Now                         1
Free to Use                         1
Name: count, dtype: int64


In [45]:
df_games.loc[df_games['price'].apply(lambda x: isinstance(x, str)), 'price'] = 0

In [46]:
df_games.isna().sum()

app_name                     0
tags                         0
price                        0
id                           0
developer                    0
release_year                 0
Accounting                   0
Action                       0
Adventure                    0
Animation &amp; Modeling     0
Audio Production             0
Casual                       0
Design &amp; Illustration    0
Early Access                 0
Education                    0
Free to Play                 0
Indie                        0
Massively Multiplayer        0
Photo Editing                0
RPG                          0
Racing                       0
Simulation                   0
Software Training            0
Sports                       0
Strategy                     0
Utilities                    0
Video Production             0
Web Publishing               0
dtype: int64

In [47]:
games = df_games.to_csv("games.csv", index=False)

In [48]:
games = pd.read_csv("games.csv")

table = pa.Table.from_pandas(games)

pq.write_table(table, "games.parquet")

In [3]:
filas1 = []
with open("australian_users_items.json", "r", encoding="Latin_1") as archivo:
    for linea in archivo:
        filas1.append(ast.literal_eval(linea))




In [4]:
df_items = pd.DataFrame(filas1)
df_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [5]:
df_items = df_items.explode("items").reset_index()

In [6]:
df_items.drop(columns= "index", inplace = True)

In [7]:
df_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '10', 'item_name': 'Counter-Strike..."
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '20', 'item_name': 'Team Fortress ..."
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '30', 'item_name': 'Day of Defeat'..."
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '40', 'item_name': 'Deathmatch Cla..."
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '50', 'item_name': 'Half-Life: Opp..."
...,...,...,...,...,...
5170010,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '373330', 'item_name': 'All Is Dus..."
5170011,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '388490', 'item_name': 'One Way To..."
5170012,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '521570', 'item_name': 'You Have 1..."
5170013,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '519140', 'item_name': 'Minds Eyes..."


In [8]:
df_items = pd.concat([df_items, pd.json_normalize(df_items["items"])], axis = 1)

In [9]:
df_items.head(10)


,user_id,items_count,steam_id,user_url,items,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '10', 'item_name': 'Counter-Strike...",10,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '20', 'item_name': 'Team Fortress ...",20,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '30', 'item_name': 'Day of Defeat'...",30,Day of Defeat,7.0,0.0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '40', 'item_name': 'Deathmatch Cla...",40,Deathmatch Classic,0.0,0.0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '50', 'item_name': 'Half-Life: Opp...",50,Half-Life: Opposing Force,0.0,0.0
5,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '60', 'item_name': 'Ricochet', 'pl...",60,Ricochet,0.0,0.0
6,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '70', 'item_name': 'Half-Life', 'p...",70,Half-Life,0.0,0.0
7,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '130', 'item_name': 'Half-Life: Bl...",130,Half-Life: Blue Shift,0.0,0.0
8,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '300', 'item_name': 'Day of Defeat...",300,Day of Defeat: Source,4733.0,0.0
9,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '240', 'item_name': 'Counter-Strik...",240,Counter-Strike: Source,1853.0,0.0


In [10]:
df_items.isna().sum()

user_id                 0
items_count             0
steam_id                0
user_url                0
items               16806
item_id             16806
item_name           16806
playtime_forever    16806
playtime_2weeks     16806
dtype: int64

In [11]:
df_items = df_items.dropna()

In [12]:
df_items.isna().sum()

user_id             0
items_count         0
steam_id            0
user_url            0
items               0
item_id             0
item_name           0
playtime_forever    0
playtime_2weeks     0
dtype: int64

In [13]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5153209 entries, 0 to 5170013
Data columns (total 9 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          object 
 3   user_url          object 
 4   items             object 
 5   item_id           object 
 6   item_name         object 
 7   playtime_forever  float64
 8   playtime_2weeks   float64
dtypes: float64(2), int64(1), object(6)
memory usage: 393.2+ MB


In [14]:
df_items.drop(columns = ["user_url", "items"], axis = 1, inplace= True)

In [15]:
df_items.head(10)

,user_id,items_count,steam_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,10,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,76561197970982479,30,Day of Defeat,7.0,0.0
3,76561197970982479,277,76561197970982479,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,277,76561197970982479,50,Half-Life: Opposing Force,0.0,0.0
5,76561197970982479,277,76561197970982479,60,Ricochet,0.0,0.0
6,76561197970982479,277,76561197970982479,70,Half-Life,0.0,0.0
7,76561197970982479,277,76561197970982479,130,Half-Life: Blue Shift,0.0,0.0
8,76561197970982479,277,76561197970982479,300,Day of Defeat: Source,4733.0,0.0
9,76561197970982479,277,76561197970982479,240,Counter-Strike: Source,1853.0,0.0


In [16]:
items = df_items.to_csv('items.csv', index = False)

In [17]:
items = pd.read_csv('items.csv')


In [18]:
print(items['item_id'])

0              10
1              20
2              30
3              40
4              50
            ...  
5153204    346330
5153205    373330
5153206    388490
5153207    521570
5153208    519140
Name: item_id, Length: 5153209, dtype: int64


In [19]:
items['item_id'] = items['item_id'].astype(int)

In [21]:
table1 = pa.Table.from_pandas(items)



In [23]:
pq.write_table(table1, "items.parquet")

In [24]:
table = pq.read_table('items.parquet')

items_parquet = table.to_pandas()

items_parquet

,user_id,items_count,steam_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,10,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,76561197970982479,30,Day of Defeat,7.0,0.0
3,76561197970982479,277,76561197970982479,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,277,76561197970982479,50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...,...,...
5153204,76561198329548331,7,76561198329548331,346330,BrainBread 2,0.0,0.0
5153205,76561198329548331,7,76561198329548331,373330,All Is Dust,0.0,0.0
5153206,76561198329548331,7,76561198329548331,388490,One Way To Die: Steam Edition,3.0,3.0
5153207,76561198329548331,7,76561198329548331,521570,You Have 10 Seconds 2,4.0,4.0


In [9]:
filas = list()

with open("australian_user_reviews.json" , "r", encoding="Latin-1") as archivo:
    for linea in archivo.readlines():
        filas.append(ast.literal_eval(linea))

df_reviews = pd.DataFrame(filas)
df_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [10]:
df_reviews = df_reviews.explode("reviews").reset_index()



In [11]:
df_reviews = pd.concat([df_reviews, pd.json_normalize(df_reviews['reviews'])], axis= 1 )


df_reviews.head(10)

,index,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review
0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011....",,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011...",,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....",,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2...",,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
5,1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted November 29, 2...",,"Posted November 29, 2013.",,239030,1 of 4 people (25%) found this review helpful,True,Very fun little game to play when your bored o...
6,2,evcentric,http://steamcommunity.com/id/evcentric,"{'funny': '', 'posted': 'Posted February 3.', ...",,Posted February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
7,2,evcentric,http://steamcommunity.com/id/evcentric,"{'funny': '', 'posted': 'Posted December 4, 20...",,"Posted December 4, 2015.","Last edited December 5, 2015.",370360,No ratings yet,True,"""Run for fun? What the hell kind of fun is that?"""
8,2,evcentric,http://steamcommunity.com/id/evcentric,"{'funny': '', 'posted': 'Posted November 3, 20...",,"Posted November 3, 2014.",,237930,No ratings yet,True,"Elegant integration of gameplay, story, world ..."
9,2,evcentric,http://steamcommunity.com/id/evcentric,"{'funny': '', 'posted': 'Posted October 15, 20...",,"Posted October 15, 2014.",,263360,No ratings yet,True,"Random drops and random quests, with stat poin..."


In [37]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415163 entries, 0 to 415162
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   level_0   415163 non-null  int64 
 1   index     415163 non-null  int64 
 2   user_id   415163 non-null  object
 3   user_url  415163 non-null  object
 4   reviews   415135 non-null  object
dtypes: int64(2), object(3)
memory usage: 15.8+ MB


In [53]:
nuls3 = df_reviews.isna().sum()

In [54]:
nuls3

index              0
user_id            0
user_url           0
reviews           28
funny             28
posted            28
last_edited       28
item_id           28
helpful           28
recommend         28
review            28
año            10147
dtype: int64

In [51]:
df_reviews.head(10)

,index,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review
0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011....",,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011...",,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....",,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2...",,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
5,1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted November 29, 2...",,"Posted November 29, 2013.",,239030,1 of 4 people (25%) found this review helpful,True,Very fun little game to play when your bored o...
6,2,evcentric,http://steamcommunity.com/id/evcentric,"{'funny': '', 'posted': 'Posted February 3.', ...",,Posted February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
7,2,evcentric,http://steamcommunity.com/id/evcentric,"{'funny': '', 'posted': 'Posted December 4, 20...",,"Posted December 4, 2015.","Last edited December 5, 2015.",370360,No ratings yet,True,"""Run for fun? What the hell kind of fun is that?"""
8,2,evcentric,http://steamcommunity.com/id/evcentric,"{'funny': '', 'posted': 'Posted November 3, 20...",,"Posted November 3, 2014.",,237930,No ratings yet,True,"Elegant integration of gameplay, story, world ..."
9,2,evcentric,http://steamcommunity.com/id/evcentric,"{'funny': '', 'posted': 'Posted October 15, 20...",,"Posted October 15, 2014.",,263360,No ratings yet,True,"Random drops and random quests, with stat poin..."


In [12]:
df_reviews["año"] = df_reviews['posted'].str.extract(r'(\d{4})')

In [13]:
df_reviews["año"].isna().value_counts()

año
False    49186
True     10147
Name: count, dtype: int64

In [14]:
df_reviews.isnull().sum()

index              0
user_id            0
user_url           0
reviews           28
funny             28
posted            28
last_edited       28
item_id           28
helpful           28
recommend         28
review            28
año            10147
dtype: int64

In [15]:
df_reviews = df_reviews.dropna(subset= ["año"])

df_reviews.head(5)

,index,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review,año
0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011
1,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011....",,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,2011
2,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011...",,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,2011
3,1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....",,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2014
4,1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2...",,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,2013


In [16]:
df_reviews.drop( columns= "posted", inplace = True)

In [17]:
df_reviews.isnull().sum()

index          0
user_id        0
user_url       0
reviews        0
funny          0
last_edited    0
item_id        0
helpful        0
recommend      0
review         0
año            0
dtype: int64

In [4]:
import vaderSentiment

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [18]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def analyze_sentiment(review):
    if isinstance(review, str):  # Checking if review is a string
        sentiment = analyzer.polarity_scores(review)

        if sentiment['compound'] >= 0.05:
            return 2  # Positive sentiment
        elif sentiment['compound'] <= -0.05:
            return 0  # Negative sentiment
        else:
            return 1  # Neutral sentiment
    else:
        return 1  # If review is not a string, consider it as neutral sentiment

df_reviews['sentiment_analysis'] = df_reviews['review'].apply(analyze_sentiment)

In [19]:
df_reviews["sentiment_analysis"].value_counts()

sentiment_analysis
2    31657
1     9926
0     7603
Name: count, dtype: int64

In [23]:
df_reviews.drop(columns = ["review", "user_url", "funny", "last_edited", "reviews"], axis= 1, inplace=True)

In [24]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49186 entries, 0 to 59304
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   index               49186 non-null  int64 
 1   user_id             49186 non-null  object
 2   item_id             49186 non-null  object
 3   helpful             49186 non-null  object
 4   recommend           49186 non-null  object
 5   año                 49186 non-null  object
 6   sentiment_analysis  49186 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 3.0+ MB


In [25]:
revies = df_reviews.to_csv('reviews.csv', index=False)


In [27]:
reviews = pd.read_csv("reviews.csv")

In [28]:
reviews = pd.read_csv('reviews.csv')

table = pa.Table.from_pandas(reviews)

pq.write_table(table, 'reviews.parquet')

In [29]:
table = pq.read_table('reviews.parquet')

reviews_parquet = table.to_pandas()

reviews_parquet

,index,user_id,item_id,helpful,recommend,año,sentiment_analysis
0,0,76561197970982479,1250,No ratings yet,True,2011,2
1,0,76561197970982479,22200,No ratings yet,True,2011,2
2,0,76561197970982479,43110,No ratings yet,True,2011,2
3,1,js41637,251610,15 of 20 people (75%) found this review helpful,True,2014,2
4,1,js41637,227300,0 of 1 people (0%) found this review helpful,True,2013,2
...,...,...,...,...,...,...,...
49181,25764,wayfeng,730,1 of 1 people (100%) found this review helpful,True,2015,1
49182,25765,76561198251004808,253980,No ratings yet,True,2015,2
49183,25769,72947282842,730,No ratings yet,True,2015,0
49184,25771,ApxLGhost,730,No ratings yet,True,2015,2
